<a href="https://colab.research.google.com/github/anthonyvann/Corporate-Analytics-in-Pandas/blob/main/Corporate_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import Random Module and Define Global Constants

In [ ]:
# Import the random module
import random as r


# Define global constants indicated by uppercased variables
ITEM_NAMES = [
    "Tablet",
    "Laptop",
    "Monitor",
    "Smartphone",
    "Smartwatch",
    "DesktopComputer",
    "GamingConsole",
    "VirtualRealityHeadset",
    "WirelessEarbuds",
    "BluetoothSpeaker",
    "DigitalCamera",
    "ActionCamera",
    "Drone",
    "PortableCharger",
    "FitnessTracker",
    "SmartHomeAssistant",
    "WirelessRouter",
    "ExternalHard Drive",
    "GraphicsTablet"
]
ADJECTIVES = [
    'Portable',
    'Innovative',
    'Sleek',
    'Durable',
    'Powerful',
    'Compact',
    'Versatile',
    'Intuitive',
    'Efficient',
    'High-resolution',
    'Wireless',
    'Smart',
    'Lightweight',
    'Rugged',
    'Affordable',
    'Fast',
    'Advanced',
    'User-friendly',
    'Reliable',
    'Ergonomic'
]
# List of first names for a Customer.
FIRST_NAMES = [
    "James", "Mary", "John", "Patricia", "Robert",
    "Jennifer", "Michael", "Linda", "William", "Elizabeth",
    "David", "Barbara", "Richard", "Susan", "Joseph",
    "Jessica", "Thomas", "Sarah", "Charles", "Karen",
    "Christopher", "Nancy", "Daniel", "Lisa", "Matthew",
    "Margaret", "Anthony", "Betty", "Mark", "Sandra",
    "Donald", "Ashley", "Steven", "Kimberly", "Paul"
]
# List of names of 10 different Stores.
STORE_NAMES = [
    "Best Buy",
    "Circuit City",
    "RadioShack",
    "Fry's Electronics",
    "Micro Center",
    "B&H Photo Video",
    "Newegg",
    "TigerDirect",
    "The Good Guys",
    "JB Hi-Fi"
]

##Define the Purchase Class

In [ ]:
class Purchase:
    # Initialize a single attribute, total, to represent the total amount
    def __init__(self):
        self.total = 0

    def increment(self, price):
        self.total += price

    # def __add__(self, other):
    #     try:
    #         # If other is an instance of int or float
    #         if isinstance(other, int) or isinstance(other, float):
    #             return self.total + other
    #         # If other is another instance of Purchase
    #         elif isinstance(other, Purchase):
    #             return self.total + other.total
    #         else:
    #             raise TypeError
    #     except TypeError:
    #         print(f"TypeError: Expected int, float, or Purchase. Received {type(other)}.")

    # def __iadd__(self, other):
    #     try:
    #         # If other is an instance of int or float
    #         if isinstance(other, int) or isinstance(other, float):
    #             self.total += other
    #         # If other is another instance of Purchase
    #         elif isinstance(other, Purchase):
    #             self.total += other.total
    #         else:
    #             raise TypeError
    #     except TypeError:
    #         print(f"TypeError: Expected int, float, or Purchase. Received {type(other)}.")

    def __str__(self):
        return f"Sales: ${self.total}"

##Define the Product Class

In [ ]:
class Product:
    def __init__(self, item_type="", item_name=""):
        # Initialize item_type & item_name with a random adjective
        # and/or item name if either arguments are empty strings.
        self.item_type = r.choice(ADJECTIVES) if not item_type else item_type
        self.item_name = r.choice(ITEM_NAMES) if not item_name else item_name
        self.price = round(r.uniform(50, 250), 2)

    # Function to return the name of the product
    def __str__(self):
        return f"Type: {self.item_type}, Item: {self.item_name}"

##Testing the Product class

In [ ]:
# Testing the class instance
test_product = Product()

# Testing __str__ with no name set
print("Testing __str__..")
print(f"test_product is: {test_product}\n")

random_name = create_name()

# Testing set_name WITH arg
print("Testing set_name with arg..")
test_product.set_name(random_name)

# Testing set_name WITHOUT arg
try:
    print("\nTesting set_name with no arg..")
    test_product.set_name()
except TypeError as err:
    print(err)

# Testing get_name
print("\nTesting get_name..")
print(test_product.get_name())

# Testing __str__ with name set
print(f"\n{test_product}")

Testing __str__..


ValueError: not enough values to unpack (expected 3, got 0)

##Define the Inventory Class

In [ ]:
class Inventory:
    def __init__(self):
        # Initialize a list of randomly generated products.
        self.inventory = [
            Product() for _ in range(r.randint(10, 20))
        ]

    # Method to display all Products in the Inventory.
    def __str__(self):
        inv_str = ""                          # String to build and return
        inv_size = self.size()                # Current size of Inventory
        if inv_size == 0:                     # Check inventory length
            return "Inventory is empty."

        for i in range(inv_size):             # Building the string
            product = str(self.inventory[i])  # Cast Product to str
            if i != inv_size - 1:             # if not last iteration
                product += '\n'               # Concat new line
            inv_str += product
        return inv_str                        # Return the result

    # Method to append a Product to the Inventory.
    def add_item(self, item: Product):
        if type(item) == Product:
            self.inventory.append(item)
        else:
            print("Only Products can be added to inventory.")

    # Method to return the size of Inventory.
    def size(self):
        return len(self.inventory)

## Testing the Inventory Class

In [ ]:
# Constant to control Inventory size
SIZE = 50

# Instance of Inventory
test_inventory = Inventory()

print("Testing __str__ with empty inventory...")
print(test_inventory)

print("\nTesting __str__ with non-empty Inventory...")
test_inventory.add_item(create_product())
print(test_inventory)

print(f"\nTesting __str__ with {SIZE} products in Inventory...")
for _ in range(SIZE):
    item = create_product()
    test_inventory.add_item(item)
print(f"{test_inventory}")

##Define the Customer Class

In [ ]:
class Customer:
    # Initialize Customer with a name.
    def __init__(self, name: str):
        self.name = name
        self.basket = Basket()

    # Method to return the name of the Customer.
    def get_name(self):
        return self.name

    # Add a Product to the basket.
    def add_to_basket(self, product):
        self.basket.add_item(product)

    # Return name of the Customer along with their basket.
    def __str__(self):
        return f"{self.name} ({self.basket})"

##Define the Store Class


In [ ]:
class Store:
    # Initialize 3 attributes: name, customers, stock
    def __init__(self, name: str):
        self.name = name          # String representing the Store name
        self.stock = Inventory()  # Inventory of Products in-stock
        self.customers = []       # List of Customers currently shopping
        self.sales = Purchase()   # Keep track of current sales of the store.

    # Return the name and the sale object of the Store.
    def __str__(self):
        return self.name

    # Method to read and return the current inventory data.
    def read_inventory(self) -> list[Product]:
        return self.stock.inventory

    # Add a Customer to the current list of customers in the Store.
    def add_customer(self, customer: Customer):
        self.customers.append(customer)

    # Method to return the current list of Customers in the Store.
    def get_customers(self) -> list[Customer]:
        for customer in self.customers:
            print(f"{customer} is currently shopping..")
        return self.customers

    # Return the total number of Customers currently in the Store.
    def customer_total(self) -> int:
        return len(self.customers)

    # Method to run a Store simulation.
    def run(self):
        print(f"Hello! Welcome to {self.name}!")
        print(f"Here is the list of products in-stock:")
        print(f"{self.stock}\n")

        print(f"There are {self.customer_total()} customers today.")

        # Iterate over the list of customers.
        for customer in self.customers:
            print(f"{customer.name} is currently shopping.")

            # Take 1 item from the inventory and add it to the customer's basket
            item = r.choice(self.read_inventory())
            customer.add_to_basket(item)
            print(f"{customer.name} added [{item}] to their basket.")

            # Increment the current sales with the price of item sold.
            self.sales.increment(item.price)
            print(f"{customer.name} purchased [{item}] for ${item.price}.")
        print(f"{self.name} Current Total {self.sales}.")


##Testing the Store Class

In [ ]:
# Instance of Store
test_store = Store("Radio Shack")

print("Testing read_inventory method with an empty inventory...")
current_inventory = test_store.read_inventory()

# Generating a single random Product
random_product = create_product()

print("\nTesting the add_item Inventory method on the 'stock' member variable of Store...")
test_store.stock.add_item(random_product)

print("\nTesting read_inventory method with a non-empty stock...")
test_store.read_inventory()

# Adding 20 more products to the inventory of test_store
for _ in range(20):
    test_store.stock.add_item(create_product())

print("\nCalling the read_inventory method with 21 products in stock...")
current_stock = test_store.read_inventory()

print("\nTesting whether the object returned from read_inventory is a list...")
print(type(current_stock))

print(f"\nTesting __str__ on each Product from the list returned from read_inventory...")
for product in current_stock:
    print(product)

##Testing Store & Customer Classes

In [ ]:
# List of customer names
CUSTOMER_NAMES = [
    "Ethan", "Hieu", "Amanda",
    "Javi", "Connie", "Zuko",
    "Suki", "Andrew", "Sharon",
    "Leyna", "Timmy", "Anthony"
]

store = Store("Microcenter")

print("Testing __str__ of Store..")
print(store)

print("\nStocking the store with 20 Products..")
for _ in range(20):
    store.stock.add_item(create_product())

print(f"\nItems in stock at {store}:")
for product in store.read_inventory():
    print(product)

# Name for the first Customer
customer_name = CUSTOMER_NAMES[-1]

print(f"\nCreating the first Customer...")
customer1 = Customer(customer_name)

print(f"\nAdding {customer1} to {store}'s list of current shoppers..")
store.add_customer(customer1)

print(f"\nAdding the remaining {len(CUSTOMER_NAMES) - 1} Customers to the Store..")
for i in range(len(CUSTOMER_NAMES) - 1):
    name = CUSTOMER_NAMES[i]
    store.add_customer(Customer(name))

print("\nTesting printability of Customer using get_customers() method..")
for customer in store.get_customers():
    print(customer)

##Testing Store & Customer Classes (cont...)

In [ ]:
# Utility function to return a Store's information as a string.
# Store information includes name, products in-stock, and customers.
def get_info(store: Store) -> str:
    info = f"STORE NAME\n• {store}\n\nPRODUCTS IN STOCK"
    info += f"\n• {store}\n"
    info += "\nPRODUCTS IN STOCK"

    for product in store.read_inventory():
        info += f"\n• {product}"

    info += "\n\nCURRENT SHOPPERS"
    for customer in store.get_customers():
        info += f"\n• {customer}"

    return info

# List of store names
STORE_NAMES = [
    "Microcenter",
    "Best Buy",
    "Radio Shack",
    "Fry's",
    "Costco",
    "Walmart",
    "Target"
]

# List of Store objects using list comprehension on STORE_NAMES
STORES = [Store(name) for name in STORE_NAMES]

# Testing printability of Store objects in stores_list
print(f"The names of our {len(STORES)} stores are:")
for store in STORES:
    print(store)
print()

# Stock up each Store with 15 Products and a group of Customers
for store in STORES:
    for _ in range(15):
        store.stock.add_item(create_product())

    print(f"Adding customers to {store}...")
    for name in CUSTOMER_NAMES:
        store.add_customer(Customer(name))
    print()
print()

# Return the information of a random store as a string
store_info = get_info(r.choice(STORES))
print(store_info)

##Define the Basket Class

In [ ]:
class Basket:
    # Initialize a list for storing Products.
    def __init__(self):
        self.products = []

    # Method to add a product to the Basket.
    def add_item(self, product: Product):
        self.products.append(product)

    # Returns the list of products in the Basket.
    def __str__(self):
        result = "Basket:\n"

        if not self.products:                # If the basket is empty
            return result + "empty"

        for i in range(len(self.products)):
            product = str(self.products[i])  # Cast Product to String
            result += product                # Concat the product to the result
            if i != len(self.products) - 1:  # Concat '\n' only if not  last iteration
                result += '\n'
        return result

## Testing the Purchase Class

##Testing via Store Simulation

In [ ]:
# Instance of Store using a randomly chosen store name.
store = Store(r.choice(STORE_NAMES))

# Iterate through the list of first names and adding them as a Customer to the store.
for name in FIRST_NAMES:
    store.add_customer(Customer(name))

store.run()

Hello! Welcome to Micro Center!
Here is the list of products in-stock:
Type: Lightweight, Item: DesktopComputer
Type: Affordable, Item: SmartHomeAssistant
Type: Reliable, Item: BluetoothSpeaker
Type: Intuitive, Item: VirtualRealityHeadset
Type: Reliable, Item: Tablet
Type: Affordable, Item: WirelessEarbuds
Type: Reliable, Item: Laptop
Type: Fast, Item: VirtualRealityHeadset
Type: Smart, Item: PortableCharger
Type: Efficient, Item: GraphicsTablet
Type: High-resolution, Item: GamingConsole
Type: Lightweight, Item: VirtualRealityHeadset
Type: Powerful, Item: SmartHomeAssistant
Type: Ergonomic, Item: VirtualRealityHeadset
Type: Efficient, Item: Smartphone

There are 35 customers today.
James is currently shopping.
James added [Type: Smart, Item: PortableCharger] to their basket.
James purchased [Type: Smart, Item: PortableCharger] for $62.56.
Mary is currently shopping.
Mary added [Type: Reliable, Item: Laptop] to their basket.
Mary purchased [Type: Reliable, Item: Laptop] for $89.7.
John 